In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [11]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [8]:
df=pd.read_csv("data/stud.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## preparing X and Y (dependent and independent feature)

In [9]:
X=df.drop(columns=['math_score'],axis=1)
## Dropping the math_score for independent feature for making output

X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [10]:
y=df['math_score']
## Dependent feature for making output
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [12]:
## Creating column transformer with 3 types of transformers

num_feature= X.select_dtypes(exclude='object').columns
cat_feature=X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder ,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
categorical_transformer=OneHotEncoder()

preprocessor=ColumnTransformer([
    ('onehotEncoder',categorical_transformer,cat_feature),
    ('standerdscaler',numeric_transformer,num_feature)
])

In [13]:
X= preprocessor.fit_transform(X)

In [14]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [15]:
X.shape

(1000, 19)

In [18]:
## spererae the datset form testing and training
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

X_train.shape ,y_test.shape

((800, 19), (200,))

## creatign Evaluation |

In [19]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_squared=r2_score(true,predicted)
    return mae,mse,rmse,r2_squared

In [20]:
models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "SVR": SVR(),
    "CatBoostRegressor": CatBoostRegressor(),
    "XGBRegressor": XGBRegressor()
    
}



model_list=[]
r2_list=[]

for i in range(len(models)):
    model= list(models.values())[i]
    model.fit(X_train,y_train)  ## Traning the model

    ##Make predictions
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    ## Evaluate Train and Test dataset
    model_train_mae,model_train_mse,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)

    model_test_mae,model_test_mse,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    r2_list.append(model_test_r2)
    
    print('Model Performance for Training set')
    print("Root Mean Squared Error: {}".format(model_train_rmse))
    print("mean_absolute_error: {}".format(model_train_mae))
    print("R2 Score: {}".format(model_train_r2))
    print("\n")
    



LinearRegression
Model Performance for Training set
Root Mean Squared Error: 5.323050852720514
mean_absolute_error: 4.266711846071957
R2 Score: 0.8743172040139593


Lasso
Model Performance for Training set
Root Mean Squared Error: 6.593815587795566
mean_absolute_error: 5.206302661246526
R2 Score: 0.8071462015863456


Ridge
Model Performance for Training set
Root Mean Squared Error: 5.323324922741654
mean_absolute_error: 4.264987823725981
R2 Score: 0.8743042615212909


KNeighborsRegressor
Model Performance for Training set
Root Mean Squared Error: 5.707683417990174
mean_absolute_error: 4.516749999999999
R2 Score: 0.8554978341651085


DecisionTreeRegressor
Model Performance for Training set
Root Mean Squared Error: 0.2795084971874737
mean_absolute_error: 0.01875
R2 Score: 0.9996534669718089


RandomForestRegressor
Model Performance for Training set
Root Mean Squared Error: 2.3269004299950415
mean_absolute_error: 1.8431166666666663
R2 Score: 0.9759834730975349


AdaBoostRegressor
Model Pe

In [21]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','R2 Score']).sort_values(by='R2 Score',ascending=False)

,Model Name,R2 Score
2,Ridge,0.880593
0,LinearRegression,0.880433
5,RandomForestRegressor,0.853377
8,CatBoostRegressor,0.851632
6,AdaBoostRegressor,0.849971
9,XGBRegressor,0.827797
1,Lasso,0.825320
3,KNeighborsRegressor,0.783813
4,DecisionTreeRegressor,0.755176
7,SVR,0.728600
